In [2]:
#프로세스 이용한 예제
import multiprocessing as mp

def washer(dishes, output): #한 대의 식기세척기 / 첫째인자 : 리스트, 둘쨰:큐
    for dish in dishes:
        print('Washing', dish, 'dish')
        output.put(dish) #결과는 output이라는 출력 큐에 전달(put이 전달 역할)
        
def dryer(input): #여러 대의 건조기
    while True:
        dish = input.get()
        print('Drying', dish, 'dish')
        input.task_done()
        
dish_queue = mp.JoinableQueue() #큐 지정
dryer_proc = mp.Process(target = dryer, args = (dish_queue,)) #dryer함수 병력처리
dryer_proc.daemon = True #스크립트가 끝날 때 모든 서브프로세스를 죽임
dryer_proc.start() #프로세스 시작

dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join() #프로세스 닫기


Washing salad dish
Washing bread dish
Washing entree dish
Washing dessert dish
Drying salad dish
Drying bread dish
Drying entree dish
Drying dessert dish


In [8]:
#스레드 이용한 예제
import threading

def do_this(what):
    whoami(what)
    
def whoami(what):
    print("Thread %s says: %s"% (threading.current_thread(), what))
    
if __name__ == "__main__":
    whoami("I'm the main program")
    for n in range(4):
        p = threading.Thread(target = do_this, 
                             args = ("I'm function %s"%n,))
        p.start()

Thread <_MainThread(MainThread, started 139739806988032)> says: I'm the main program
Thread <Thread(Thread-20, started 139739218822912)> says: I'm function 0
Thread <Thread(Thread-21, started 139739210430208)> says: I'm function 1
Thread <Thread(Thread-22, started 139739218822912)> says: I'm function 2
Thread <Thread(Thread-23, started 139739210430208)> says: I'm function 3


In [16]:
import threading, queue
import time

def washer(dishes, dish_queue):
    for dish in dishes:
        print("Washing", dish)
        time.sleep(5)
        dish_queue.put(dish)
        
def dryer(dish_queue):
    while True:
        dish = dish_queue.get()
        print("Drying", dish)
        time.sleep(10)
        dish_queue.task_done()
        
dish_queue = queue.Queue()
for n in range(2):
    dryer_thread = threading.Thread(target = dryer,
                                   args = (dish_queue,))
    dryer_thread.start()
    
dishes = ['salad', 'bread', 'entree', 'dessert']
washer(dishes, dish_queue)
dish_queue.join()

Washing salad
Washing breadDrying
 salad
WashingDrying  breadentree

Washing dessert
Drying entree
Drying dessert


In [14]:
#socket사용 예제
import gevent
from gevent import socket

hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
        'www.antique-taxidermy.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout = 5)
for job in jobs:
    print(job.value)

66.6.44.4
172.217.25.115
None


In [16]:
#monkey-patch 예제
import gevent
from gevent import socket
from gevent import monkey
monkey.patch_all() # gevent의 영향을 가능한 많이 받아서 속도 향상 되도록함

hosts = ['www.crappytaxidermy.com', 'www.walterpottertaxidermy.com',
        'www.antique-taxidermy.com']
jobs = [gevent.spawn(gevent.socket.gethostbyname, host) for host in hosts]
gevent.joinall(jobs, timeout = 5)
for job in jobs:
    print(job.value)

66.6.44.4
172.217.25.211
None


In [10]:
#redis_washer.py
import redis
conn = redis.Redis()
print('Washer is starting')
dishes = ['salad', 'bread', 'entree', 'dessert']
for dish in dishes :
    msg = dish.encode('utf-8')
    conn.rpush('dishes', msg)
    print('Washed', dish)

conn.rpush('dishes', 'quit')
print('Washer is done')


Washer is starting
Washed salad
Washed bread
Washed entree
Washed dessert
Washer is done


In [15]:
#redis_dryer.py
import redis
conn = redis.Redis()

print('Dryer is starting')
while True:
    msg = conn.blpop('dishes')
    if not msg:
        break
    val = msg[1].decode('utf-8')
    if val == 'quit':
        break
    print('Dried', val)
print('Dishes are dried')


Dryer is starting
Dried bread
Dishes are dried


In [ ]:
#redis_dryer2.py
def dryer():
    import redis
    import os
    import time
    conn = redis.Redis()
    pid = os.getpid()
    timeout = 20
    print('Dryer process %s is starting'%pid)

    while True:
        msg = conn.blpop('dishes', timeout)
        if not msg:
            break
        val = msg[1].decode('utf-8')
        if val == 'quit':
            break
        print('%s: dried %s' % (pid, val))
        time.sleep(0.1)
    print('Dryer process %s is done' %pid)

import multiprocessing
DRYERS = 3
for num in range(DRYERS):
    p = multiprocessing.Process(target = dryer)
    p.start()

